In [ ]:
!pip install googletrans==3.1.0a0
'''
Скачиваем необходимые модели 
'''
!python3 -m spacy download ru_core_news_lg
!python3 -m spacy download en_core_web_sm


In [ ]:
from googletrans import Translator
import spacy
from spacy.lang.ru.examples import sentences 
from spacy.matcher import PhraseMatcher
import pandas as pd
import re 

In [ ]:
data = pd.read_csv('/content/test_data.csv') # Исходные данные 
manager_repl = data[data['role']=='manager'] # Данные о репликах менеджера
new_data = manager_repl.groupby(['dlg_id','line_n'])['text'] # Текст каждого менеджера

repl_managers = {} # { идентификатор диалога: [реплики менеджера]}
for dlg_id, text in new_data:
  repl_managers[dlg_id] = list(text.values)


# Приветсвтия с регуляркой ГУД

Используем русскоязычную модель и набор заготовленных фраз, на основе которых, будем искать совпадения.

In [ ]:
nlp = spacy.load('ru_core_news_lg')
phrase_matcher = PhraseMatcher(nlp.vocab)

In [ ]:
greeting_phrases = ['здравствуйте','доброе утро', 'добрый день','добрый вечер', 'доброго времени суток', 'привет']
patterns = [nlp(manager_repl) for manager_repl in greeting_phrases]
phrase_matcher.add('GREET', None, *patterns)


In [ ]:
greeting_repl = {}
for key, value in repl_managers.items():
  for repl in value:
    repl_lower = repl.lower()
    sentence = nlp(repl_lower)
    matched_phrases = phrase_matcher(sentence)
    if len(matched_phrases) > 0:
      greeting_repl[key] = repl
print(greeting_repl)

{(0, 1): 'Алло здравствуйте', (1, 1): 'Алло здравствуйте', (2, 2): 'Алло здравствуйте', (3, 1): 'Алло дмитрий добрый день'}


In [ ]:
for dlg_id, line_n in greeting_repl:
  data.at[data.index[(data['line_n']==line_n) & (data['dlg_id']==dlg_id)],'greeting'] = True


# Представление с переводом ГУД

Также ищем совпадения по фразам, но уже на английском языке. Для этого перед поиска совпадения переводим реплику.

In [ ]:
introduce_phrases = ['my name is', 'iam', 'i am',]
patterns = [nlp(manager_repl) for manager_repl in introduce_phrases]
phrase_matcher.add('intro', None, *patterns)
translator = Translator()

In [ ]:
introduce_repl = {}
for key, value in repl_managers.items():
  for repl in value:
    repl_lower = repl.lower()
    translated_text = translator.translate(repl_lower, dest='en')
    sentence = nlp(translated_text.text)
    matched_phrases = phrase_matcher(sentence)
    if len(matched_phrases) > 0:
      introduce_repl[key] = repl
print(introduce_repl)

{(0, 3): 'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается', (1, 2): 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается', (2, 3): 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там', (3, 2): 'Добрый меня максим зовут компания китобизнес удобно говорить'}


In [ ]:
for dlg_id, line_n in introduce_repl:
  data.at[data.index[(data['line_n']==line_n) & (data['dlg_id']==dlg_id)],'introduce'] = True


,dlg_id,line_n,role,text,greeting,introduce
0,0,0,client,Алло,NaN,NaN
1,0,1,manager,Алло здравствуйте,True,NaN
2,0,2,client,Добрый день,NaN,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,NaN,True
4,0,4,client,Ага,NaN,NaN
...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,NaN,NaN
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,NaN,NaN
477,5,140,client,Спасибо спасибо,NaN,NaN
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,NaN,NaN


# Имена менеджеров с регуляркой ГУД
Каждую реплику прогоняем через набор паттернов в регулярном выражении.

In [ ]:
names_repl = {}
for row in manager_repl.iterrows():
  repl = row[1]['text']
  dlg_id = row[1]['dlg_id']
  line_n = row[1]['line_n']
  res = re.findall('(?<=меня зовут )\w+|(?<=зовут меня )\w+|(?<=меня )\w+(?= зовут)|\w+(?= меня зовут)', repl.lower())
  names_repl[(dlg_id,line_n)] = res


In [ ]:
for dlg_id, line_n in names_repl:
  if len(names_repl[(dlg_id, line_n)]) > 0:
    data.at[data.index[(data['line_n']==line_n) & (data['dlg_id']==dlg_id)],'manager_name'] = names_repl[(dlg_id, line_n)]


,dlg_id,line_n,role,text,greeting,introduce,manager_name
0,0,0,client,Алло,NaN,NaN,NaN
1,0,1,manager,Алло здравствуйте,True,NaN,NaN
2,0,2,client,Добрый день,NaN,NaN,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,NaN,True,ангелина
4,0,4,client,Ага,NaN,NaN,NaN
...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,NaN,NaN,NaN
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,NaN,NaN,NaN
477,5,140,client,Спасибо спасибо,NaN,NaN,NaN
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,NaN,NaN,NaN


# Прощания с переводом ГУД

Также ищем совпадения по фразам, но уже на английском языке. Для этого перед поиска совпадения переводим реплику.

In [ ]:
nlp = spacy.load('en_core_web_sm')
phrase_matcher = PhraseMatcher(nlp.vocab)

bye_phrases = ['goodbye','have a good day']
patterns = [nlp(manager_repl) for manager_repl in bye_phrases]
phrase_matcher.add('goodb', None, *patterns)

repl_managers = {}
for state, frame in new_data:
  repl_managers[state] = list(frame.values)

In [ ]:
goodbye_repl = {}
for key, value in repl_managers.items():
  for repl in value:
    repl_lower = repl.lower()
    translated_text = translator.translate(repl_lower, dest='en')
    sentence = nlp(translated_text.text)
    matched_phrases = phrase_matcher(sentence)
    if len(matched_phrases) > 0:
      goodbye_repl[key] = repl


print(goodbye_repl)

{(0, 108): 'Всего хорошего до свидания', (1, 54): 'До свидания', (4, 33): 'Во вторник все ну с вами да тогда до вторника до свидания', (5, 142): 'Ну до свидания хорошего вечера'}


In [ ]:
for dlg_id, line_n in goodbye_repl:
  if len(goodbye_repl[(dlg_id, line_n)]) > 0:
    data.at[data.index[(data['line_n']==line_n) & (data['dlg_id']==dlg_id)],'goodbye'] = True
data

,dlg_id,line_n,role,text,greeting,introduce,manager_name,goodbye
0,0,0,client,Алло,NaN,NaN,NaN,NaN
1,0,1,manager,Алло здравствуйте,True,NaN,NaN,NaN
2,0,2,client,Добрый день,NaN,NaN,NaN,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,NaN,True,ангелина,NaN
4,0,4,client,Ага,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,NaN,NaN,NaN,NaN
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,NaN,NaN,NaN,NaN
477,5,140,client,Спасибо спасибо,NaN,NaN,NaN,NaN
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,NaN,NaN,NaN,NaN


# Извлечение имени менеджера SO SO

In [ ]:
!pip install natasha

In [ ]:
import pandas as pd
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [ ]:
data = pd.read_csv('/content/test_data.csv')
text = data[data['role']=='manager']['text'].values


In [ ]:
morph_vocab = MorphVocab()
extractor = NamesExtractor(morph_vocab)
names = []
# print(text)
for repl in text:
  matches = extractor.find(repl)
  if matches != None:
    names.append(matches.fact.first)
    print(f'text = {matches.fact}')

print(names)

# Компания

Вытягиваем именованные сущности на английском языке и также с помощью регулярного выражения.

In [ ]:
translator = Translator()
nlp = spacy.load("en_core_web_sm")  # load the English model

companies_en = {}
for row in data.iterrows():
  repl = row[1]['text']
  dlg_id = row[1]['dlg_id']
  line_n = row[1]['line_n']
  translated_text =  translator.translate(repl, dest='en')
  doc = nlp(translated_text.text)
  for ent in doc.ents:
    if ent.label_ == 'ORG':
      companies_en[(dlg_id,line_n)] = ent.text.strip()
      print(ent.text.strip(), ent.label_)


Digital ORG
Iraklas dot ORG
Zhesupov ORG
WhatsApp ORG
WhatsApp ORG
Aidar ORG
Dimin ORG


In [ ]:

for dlg_id, line_n in companies_en:
  if len(companies_en[(dlg_id, line_n)]) > 0:
    data.at[data.index[(data['line_n']==line_n) & (data['dlg_id']==dlg_id)],'company_name_(en)'] = companies_en[(dlg_id, line_n)]

data

,dlg_id,line_n,role,text,greeting,introduce,manager_name,goodbye,company_name_(en)
0,0,0,client,Алло,NaN,NaN,NaN,NaN,NaN
1,0,1,manager,Алло здравствуйте,True,NaN,NaN,NaN,NaN
2,0,2,client,Добрый день,NaN,NaN,NaN,NaN,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,NaN,True,ангелина,NaN,NaN
4,0,4,client,Ага,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,NaN,NaN,NaN,NaN,NaN
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,NaN,NaN,NaN,NaN,NaN
477,5,140,client,Спасибо спасибо,NaN,NaN,NaN,NaN,NaN
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,NaN,NaN,NaN,NaN,NaN


In [ ]:
companies = {}

for row in data.iterrows():
  repl = row[1]['text']
  dlg_id = row[1]['dlg_id']
  line_n = row[1]['line_n']
  res = re.findall('(?<=компания )\w+ \w+', repl.lower())
  companies[(dlg_id,line_n)] = res

In [ ]:
for dlg_id, line_n in companies:
  if len(companies[(dlg_id, line_n)]) > 0:
    data.at[data.index[(data['line_n']==line_n) & (data['dlg_id']==dlg_id)],'company_name_(regex)'] = companies[(dlg_id, line_n)]
data

,dlg_id,line_n,role,text,greeting,introduce,manager_name,goodbye,company_name_(en),company_name_(regex)
0,0,0,client,Алло,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,manager,Алло здравствуйте,True,NaN,NaN,NaN,NaN,NaN
2,0,2,client,Добрый день,NaN,NaN,NaN,NaN,NaN,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,NaN,True,ангелина,NaN,NaN,диджитал бизнес
4,0,4,client,Ага,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,NaN,NaN,NaN,NaN,NaN,NaN
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,NaN,NaN,NaN,NaN,NaN,NaN
477,5,140,client,Спасибо спасибо,NaN,NaN,NaN,NaN,NaN,NaN
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,NaN,NaN,NaN,NaN,NaN,NaN


# Соответствие нормам вежливости

Проверяем, если менеджер поздоровался и попрощался в рамках одного диалога, мы помечаем его как "вежливого" и ставим пометку True

In [ ]:

repl_id = []

for id in list(greeting_repl.keys()):
  repl_id.append(id[0])

for id in list(goodbye_repl.keys()):
  repl_id.append(id[0])

repl_id.sort()
normal = []
unnormal = [] 
for id in range(len(repl_id) - 1):
  if repl_id[id] == repl_id[id+1]:
    normal.append(repl_id[id])

unnormal = list(set(repl_id) - set(normal))


In [ ]:

for row in data.iterrows():
  dlg_id = row[1]['dlg_id']
  if dlg_id in unnormal:
    data.at[data.index[(data['dlg_id']==dlg_id)],'polite'] = False
  else: 
    data.at[data.index[(data['dlg_id']==dlg_id)],'polite'] = True


data

,dlg_id,line_n,role,text,greeting,introduce,manager_name,goodbye,company_name_(en),company_name_(regex),polite
0,0,0,client,Алло,NaN,NaN,NaN,NaN,NaN,NaN,True
1,0,1,manager,Алло здравствуйте,True,NaN,NaN,NaN,NaN,NaN,True
2,0,2,client,Добрый день,NaN,NaN,NaN,NaN,NaN,NaN,True
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,NaN,True,ангелина,NaN,NaN,диджитал бизнес,True
4,0,4,client,Ага,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,NaN,NaN,NaN,NaN,NaN,NaN,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,NaN,NaN,NaN,NaN,NaN,NaN,False
477,5,140,client,Спасибо спасибо,NaN,NaN,NaN,NaN,NaN,NaN,False
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
data.to_csv('result.csv')